In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
import os
import pandas as pd
#from langchain_google_genai import ChatGoogleGenerativeAI

E:\anaconda\envs\rag_final\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_ollama import OllamaLLM

In [3]:
# 1. Загрузка той же самой модели эмбеддингов, которая использовалась для создания индекса
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

C:\Users\User\AppData\Local\Temp\ipykernel_1580\3774963524.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [4]:
#  Загрузка FAISS-индекса с диска
loaded_vectorstore_recursive = FAISS.load_local(
    "faiss_index_recursive2",  
    embeddings,               
    allow_dangerous_deserialization=True 
)

In [5]:
loaded_vectorstore_fusion = FAISS.load_local(
    "faiss_index_fusion_2",  
    embeddings,               
    allow_dangerous_deserialization=True 
)

In [6]:
retriever_recursive = loaded_vectorstore_recursive.as_retriever(search_kwargs={"k": 4})

In [7]:
retriever_fusion = loaded_vectorstore_fusion.as_retriever(search_kwargs={"k": 4})

In [8]:
'''retriever_hyde = loaded_vectorstore_hyde.as_retriever(search_kwargs={"k": 4})'''

'retriever_hyde = loaded_vectorstore_hyde.as_retriever(search_kwargs={"k": 4})'

In [9]:
# Индекс загружен и готов к поиску
query = "Какие есть вакансии для Data Scientist?"
results = loaded_vectorstore_recursive.similarity_search(query, k=4)

In [10]:
llm = OllamaLLM(model="llama3", temperature=0)

In [11]:
system_template = (
   "Ты — HR-помощник, специалист по подбору персонала.\n\n"
        "Инструкции:\n"
        "1. Отвечай ТОЛЬКО на основе предоставленного контекста\n"
        "2. Структурируй ответ на русском языке\n"
        "3. ВСЕГДА указывай источники в формате [Источник: ID_вакансии]\n"
        "4. Если информации недостаточно - прямо скажи об этом\n\n"
        "Контекст: {context}\n\n"
        "Вопрос: {question}"
)

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", "{question}"), 
    ])

In [13]:
def format_docs(docs):
    # Объединяем текст найденных чанков в одну большую строку
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
query_test = "Какие есть вакансии Data Scientist санкт петербурге?"

In [15]:
rag_chain_recursive = (
    {
        # Шаг 1: Поиск контекста (вопрос -> retriever -> форматирование)
        "context": itemgetter("question") | retriever_recursive | format_docs,  
        # Шаг 2: Сохранение исходного вопроса
        "question": itemgetter("question")
    }
    | prompt          # Шаг 3: Применение промпта к контексту и вопросу
    | llm             # Шаг 4: Передача в модель Gemini
    | StrOutputParser() # Шаг 5: Получение ответа в виде строки
)

In [16]:
response = rag_chain_recursive.invoke({"question": query_test})
response

'Вакансии Data Scientist в Санкт-Петербурге:\n\n1. Вакансия: Data Scientist (с фокусом на LLM, RAG и Агентов). Город:  Санкт-Петербург. Опыт: От 3 до 6 лет. График: Удаленная работа. Занятость: Полная занятость. Минимальная зарплата: 0.0.\n\n[ДЕТАЛЬНОЕ ОПИСАНИЕ]: высыпаться выделять время хобби льготы меры господдержки предусмотренные it компаний аккредитованы минцифры возможность разрабатывать it решения разных отраслей пробовать новые технологии выстроены процессы удаленной работы просторные офисы зонами коворкинга комфортом пообщаться поиграть настолки тишине обдумать рабочие задачи возможность прокачаться интересующих направлениях развитую систему наставничества сертификацию счёт компании участие конференциях изучение английского языка активный\n\n[Источник: ID_вакансии]\n\nВакансия: Старший Data Scientist, Построение запросов. Город:  Москва. Опыт: От 3 до 6 лет. График: Удаленная работа. Занятость: Полная занятость. Минимальная зарплата: 0.0.\n\n[ДЕТАЛЬНОЕ ОПИСАНИЕ]: который помо

In [17]:
rag_chain_fusion = (
    {
        # Шаг 1: Поиск контекста (вопрос -> retriever -> форматирование)
        "context": itemgetter("question") | retriever_fusion | format_docs,  
        # Шаг 2: Сохранение исходного вопроса
        "question": itemgetter("question")
    }
    | prompt          # Шаг 3: Применение промпта к контексту и вопросу
    | llm             # Шаг 4: Передача в модель Gemini
    | StrOutputParser() # Шаг 5: Получение ответа в виде строки
)

In [18]:
response = rag_chain_fusion.invoke({"question": query_test})
response

'Вакансии Data Scientist в Санкт-Петербурге:\n\n* Вакансия: Data Scientist / Researcher. Город: Санкт-Петербург. Опыт: От 1 года до 3 лет. График: Удаленная работа. Занятость: Полная занятость. Минимальная зарплата: 0.0. [ДЕТАЛЬНОЕ ОПИСАНИЕ]: данных генерации прототипов высшее образование области математики статистики информатики связанных областях условия возможность публикаций участия конференциях работа крупными массивами данных сложные аналитические задачи использование передовых инструментов технологий участие развитии компании возможность влиять продукт процессы компании достойная конкурентноспособная заработная плата удаленная работа периодически офлайн встречи г санкт петербург возможность профессионального развития уважаемые соискатели связи большим количеством поступивших откликов время обработки увеличено просим отнестись этому пониманием\n* Вакансия: Senior Data Scientist (Rec. system). Город: Санкт-Петербург. Опыт: От 3 до 6 лет. График: Полный день. Занятость: Полная заня